In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/mnt/hdd01/patentsview/Non Cleantech Patents - Classifier Set/df_epo_non_cleantech_text_data_abstr_claim.csv')

In [7]:
# Delete all rows where appln_lng is not 'en' and where appln_comp is not TITLE or ABSTR
df = df[(df['appln_lng'] == 'en') & (df['appln_comp'].isin(['ABSTR', 'TITLE']))]

In [9]:
df_title = df[df['appln_comp'] == 'TITLE']
df_abstr = df[df['appln_comp'] == 'ABSTR']

In [10]:
# Create a custom order for appln_kind
order = {'B9': 11, 'B8': 10, 'B3': 9, 'B2': 8, 'B1': 7, 'A9': 6, 'A8': 5, 'A4': 4, 'A3': 3, 'A2': 2, 'A1': 1}
df_title['kind_order'] = df_title['appln_kind'].map(order)
df_abstr['kind_order'] = df_abstr['appln_kind'].map(order)

# Sort and group
df_title_sorted = df_title.sort_values(by=['appln_comp', 'publn_nr', 'kind_order'], ascending=[True, True, False])
df_title_sorted = df_title_sorted.groupby(['appln_comp', 'publn_nr']).first().reset_index()

df_abstr_sorted = df_abstr.sort_values(by=['appln_comp', 'publn_nr', 'kind_order'], ascending=[True, True, False])
df_abstr_sorted = df_abstr_sorted.groupby(['appln_comp', 'publn_nr']).first().reset_index()

# Drop the kind_order column
df_title_sorted = df_title_sorted.drop(columns=['kind_order'])
df_abstr_sorted = df_abstr_sorted.drop(columns=['kind_order'])

/tmp/ipykernel_1147595/3177503836.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_title['kind_order'] = df_title['appln_kind'].map(order)
/tmp/ipykernel_1147595/3177503836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstr['kind_order'] = df_abstr['appln_kind'].map(order)


In [14]:
# Pivot dataframes on publn_nr
df_title_pivot = df_title_sorted.pivot(index='publn_nr', columns='appln_comp', values='appln_text')
df_abstr_pivot = df_abstr_sorted.pivot(index='publn_nr', columns='appln_comp', values='appln_text')

In [16]:
# Merge the two dataframes and reset index
df_merged = pd.merge(df_title_pivot, df_abstr_pivot, on='publn_nr')
df_merged = df_merged.reset_index()

In [17]:
df_merged.head()

appln_comp,publn_nr,TITLE,ABSTR
0,197,Rechargeable non-aqueous cell with a chalcogen...,"<p id=""pa01"" num=""0001"">A non-aqueous secondar..."
1,231,A distribution device for a fluid such as water,"<p id=""pa01"" num=""0001"">A distribution device ..."
2,237,Apparatus for the infusion of a labour-inducin...,"<p id=""pa01"" num=""0001"">In apparatus for the a..."
3,407,Process for applying a radiation curable coati...,"<p id=""pa01"" num=""0001"">A method is provided f..."
4,417,Suction dredger and method of loading the same,"<p id=""pa01"" num=""0001"">In a method of loading..."


In [18]:
from bs4 import BeautifulSoup
import re

def clean_abstr_text_html(abstr_text):
    # If the input is a list, join it into a string
    if isinstance(abstr_text, list):
        abstr_text = ' '.join(abstr_text)
    elif abstr_text is None:
        return None

    # Remove all instances of <!--(.*?)-->
    abstr_text = re.sub(r'<!--.*?-->', ' ', abstr_text)

    try:
        # Parse the claim_text as HTML
        soup = BeautifulSoup(abstr_text, 'html.parser')
        
        # Extract all text from <p> tags
        cleaned_texts = [elem.text for elem in soup.find_all('p') if elem.text]

        # Remove claim numbers if they exist
        cleaned_texts = [re.sub(r'^\d+\.\s*', '', text) for text in cleaned_texts]

        # Join the cleaned texts
        cleaned_text = ' '.join(cleaned_texts)

    except Exception as e:
        return abstr_text  # Return the original text if parsing fails
    
    return cleaned_text.strip()

# Clean abstr text
df_merged['ABSTR'] = df_merged['ABSTR'].apply(clean_abstr_text_html)

In [20]:
df_merged.to_csv('/mnt/hdd01/patentsview/Non Cleantech Patents - Classifier Set/df_epo_non_cleantech_text_data_abstr_cleaned.csv', index=False)